In [ ]:
%cd ../src
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
requests.packages.urllib3.disable_warnings() 

import claim_verification

## Brad pitt married to Britney Spears? (fake)

In [ ]:
claim = "brad pitt is to marry with britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "brad pitt is not married to britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## Joe Biden classified documents found? (true)

In [ ]:
claim = "Joe Biden took home classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "Joe Biden never took home any classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## COVID-19 vaccine causes infertility (fake)

In [ ]:
claim = "COVID-19 vaccine causes infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "COVID-19 vaccine does not cause infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## Jan Smit met de US president (fake, but no relevant evidence)

In [ ]:
claim = "jan smit has met with US president in 2012"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

# Try other models

In [ ]:
from sentence_transformers import CrossEncoder
from transformers import pipeline
import requests
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
import torch

requests.packages.urllib3.disable_warnings() 

import get_evidences

### Setup

In [ ]:
snli_labels = ['contradiction', 'entailment', 'neutral']

In [ ]:
claim = 'COVID-19 vaccine causes infertility'
evidence = claim_verification.load_evidence(claim)
context = evidence.text[0]

### Q&A

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
summarized_context = summarizer(context, min_length=5, max_length=50)
summarized_context = summarized_context[0]['summary_text']
summarized_context

In [ ]:
snli_labels[model.predict([(claim, summarized_context)]).argmax(axis=1)[0]]

In [ ]:
snli_labels[model.predict([claim, test[4]]).argmax()]

In [ ]:
claim = 'COVID-19 vaccine causes infertility'

In [ ]:
test[3]

In [ ]:
model.predict([claim, summarized_context])

In [ ]:
summarized_text = summarizer(context, min_length=10, max_length=128)[0]['summary_text']
sentences = summarized_text.split('.')
sentences = [sent for sent in sentences if len(sent.split(' '))>3]

print(claim)
print(snli_labels[model.predict([(claim, summarized_text)]).argmax(axis=1)[0]]+'\n')
for sent in sentences:
    print(sent)
    print(snli_labels[model.predict([(claim, sent)]).argmax(axis=1)[0]])
    print()

In [ ]:
snli_labels[model.predict([('COVID-19 vaccine does not cause infertility', 'COVID-19 vaccine causes infertility')]).argmax(axis=1)[0]]

In [ ]:
# https://huggingface.co/datasets/snli
model = CrossEncoder('cross-encoder/nli-distilroberta-base')

result = model.predict([(claim, summarized_context)])
print(result)
snli_labels[result.argmax(axis=1)[0]]

### Zero-shot

In [ ]:
from transformers import pipeline

zero_shot_model = pipeline(model="facebook/bart-large-mnli")

In [ ]:
claim1 = "Joe Biden took home classified documents after leaving the vice-presidency"
claim2 = 'documents were found in the home of Biden'

conclusion = oracle(claim2, candidate_labels=[f"{claim1} is true", f"{claim1} is false"])
print(conclusion)

In [ ]:
temp_labels = [f"{claim1} is {str(bool(0))}", f"{claim1} is {str(bool(1))}"]

claim1 = "Joe Biden took home classified documents after leaving the vice-presidency"
claim2 = 'documents were not found in the home of Biden'

conclusion = oracle(claim2, candidate_labels=temp_labels)
bool(np.argmax(conclusion['scores']))

In [ ]:
context = f"{evidence.text[0]}."
conclusion = oracle(context, candidate_labels=[f"{claim} is true", f"{claim} is false"])
print(conclusion)

In [ ]:
context = f"{evidence.text[0]}."
conclusion = oracle(context, candidate_labels=[f"{claim} is false", 
                                               f"{claim} is true"])
print(conclusion)

In [ ]:
context = f"Claim: COVID-19 vaccine does not cause infertility. Information: {summarized_context}."
conclusion = oracle(context, candidate_labels=["False claim", "True claim"])
print(conclusion)

In [ ]:
context = f"Claim: {claim}. Information: {summarized_context}."
conclusion = oracle(context, candidate_labels=["False claim", "True claim"])
print(conclusion)

In [ ]:
# claim = 'COVID-19 vaccine causes infertility'
# for ev in evidence['text'].values:
#     context = f"{claim}. This is logically supported by the following passages. {ev}"
#     conclusion = oracle(context, candidate_labels=["true", "unsure", "false"])
#     print(conclusion)

## Text Generation

In [ ]:
generator = pipeline(model="gpt2", max_length=100)

In [ ]:
generator(f"{summarized_context}. Therefore, is the claim '{claim}' true or false?", do_sample=False)

### Bevindingen

- NLI is te streng